In [8]:
import simplejson as json
import pandas as pd 
import os
import numpy as np
import sys
sys.path.append('C:\\Users\\embicks\\AppData\\Local\\Continuum\\Anaconda3\\envs\\dotce\\lib\\site-packages\\pyLDAvis')
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression as LR
from sklearn.naive_bayes import MultinomialNB

C:\Users\embicks\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [56]:
#this step loops through all json files in the directory and converts them into one df 
#some files have metadata that contains the classification and some doesn't ... still working through how to figure that out

path_to_json = 'C:/Users/embicks/Documents/DOTCE/Email_Test'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['messageId', 'subject', 'priority', 'sent_date', 'body', 'sensitivity'])

# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        exclude = set(string.punctuation)
        json_text = json.load(json_file)
        properties_list = json_text['metadata']['properties']
        messageId = json_text['metadata']['message']['messageId']
        subject = json_text['metadata']['message']['subject']
        priority = json_text['metadata']['message']['priority']
        sent_date = json_text['metadata']['message']['sentDate']
        body = json_text['metadata']['message']['plainTextBody']
        body = body.replace('\r\n','')
        sensitivity = str(next((p.values() for p in properties_list if p.get('key')== 'Sensitivity')))
        sensitivity = sensitivity.replace('dict_values','')
        sensitivity = ''.join(ch for ch in sensitivity if ch not in exclude)
        sensitivity.strip()
        jsons_data.loc[index] = [messageId, subject, priority, sent_date, body, sensitivity]
print(jsons_data)

                                            messageId  \
0    AI0QNEC5T2U4.LINXNFAF2B773@mimefactory.state.tld   
1    WK27VSP4T2U4.LA5RVB4K9PHD2@mimefactory.state.tld   
2     SFSUTM95T2U4.FV22VGDQFNW2@mimefactory.state.tld   
3    4NM1JW56T2U4.670OV7SXX1I31@mimefactory.state.tld   
4    IHZSLSX5T2U4.6L7T408HVJHM2@mimefactory.state.tld   
5    DDHP3S06T2U4.0BRH9QVYEVYX2@mimefactory.state.tld   
6     SS2YSPG6T2U4.96G15KMFPZ8Z@mimefactory.state.tld   
7     SJYDXZC6T2U4.YV3DRS5IUGER@mimefactory.state.tld   
8    YP3M7ST5T2U4.052ISXVEFH8Y2@mimefactory.state.tld   
9    LLQRKA06T2U4.58FKRD2QXS433@mimefactory.state.tld   
10    2IVX2ZX5T2U4.KITZF1YK1GK2@mimefactory.state.tld   
11   L9NMLQQ4T2U4.AXWR9RZGG0223@mimefactory.state.tld   
12    DR74CBZ4T2U4.POQIU0HT4NP3@mimefactory.state.tld   
13   XQPWNRM4T2U4.9E12WI5MXT2M3@mimefactory.state.tld   
14   YNLIAFC5T2U4.V27HU99SKF0F3@mimefactory.state.tld   
15   1AJPU7S4T2U4.9RRJTZ4W0TT41@mimefactory.state.tld   
16   LSX6JDZ4T2U4.3HG3M6KSTXUA1

                                            messageId  \
0    AI0QNEC5T2U4.LINXNFAF2B773@mimefactory.state.tld   
1    WK27VSP4T2U4.LA5RVB4K9PHD2@mimefactory.state.tld   
2     SFSUTM95T2U4.FV22VGDQFNW2@mimefactory.state.tld   
3    4NM1JW56T2U4.670OV7SXX1I31@mimefactory.state.tld   
4    IHZSLSX5T2U4.6L7T408HVJHM2@mimefactory.state.tld   
5    DDHP3S06T2U4.0BRH9QVYEVYX2@mimefactory.state.tld   
6     SS2YSPG6T2U4.96G15KMFPZ8Z@mimefactory.state.tld   
7     SJYDXZC6T2U4.YV3DRS5IUGER@mimefactory.state.tld   
8    YP3M7ST5T2U4.052ISXVEFH8Y2@mimefactory.state.tld   
9    LLQRKA06T2U4.58FKRD2QXS433@mimefactory.state.tld   
10    2IVX2ZX5T2U4.KITZF1YK1GK2@mimefactory.state.tld   
11   L9NMLQQ4T2U4.AXWR9RZGG0223@mimefactory.state.tld   
12    DR74CBZ4T2U4.POQIU0HT4NP3@mimefactory.state.tld   
13   XQPWNRM4T2U4.9E12WI5MXT2M3@mimefactory.state.tld   
14   YNLIAFC5T2U4.V27HU99SKF0F3@mimefactory.state.tld   
15   1AJPU7S4T2U4.9RRJTZ4W0TT41@mimefactory.state.tld   
16   LSX6JDZ4T2U4.3HG3M6KSTXUA1

In [71]:
email_df = jsons_data
email_df.groupby('sensitivity').count()
# Filter out empty rows
non_empty_df = email_df[email_df['body'].isnull() == False].sample(frac=.2)
#sample method chooses a random sample of the origina frame
#https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
print("Original dataframe contains {} messages\nNon-empty datafram contains {} messages\n".format(
len(email_df), len(non_empty_df)))

Original dataframe contains 358 messages
Non-empty datafram contains 72 messages



In [72]:
#Add new column to dataframe with True if cat == fraud_waste_abuse
#Edit the column names and truth conditions to match your data
non_empty_df['personal'] = (non_empty_df['sensitivity'] == 'Sensitivity Personal')
#Create a vector of class labels
class_labels = non_empty_df['personal']
#use value_counts() method of series
class_labels.value_counts()
print(non_empty_df)


                                            messageId  \
330  HP0CTGJ6T2U4.26RQ7U8N1SJN2@mimefactory.state.tld   
159  WH0HHC36T2U4.ZXPIC0IPH5PR1@mimefactory.state.tld   
202  UXQDVX26T2U4.OLEKKXACW9A51@mimefactory.state.tld   
111   2A7NE725T2U4.9FCMDVP1SLLA@mimefactory.state.tld   
157  D1ONQ1P4T2U4.6U2FL1YVWENQ3@mimefactory.state.tld   
255  K5ELBGM4T2U4.YL0AUXXTJKBT1@mimefactory.state.tld   
153  K2C7GF56T2U4.NK04AVW2I6Y03@mimefactory.state.tld   
348  Z8GY3M65T2U4.U9ZIXVFB0L1W1@mimefactory.state.tld   
10    2IVX2ZX5T2U4.KITZF1YK1GK2@mimefactory.state.tld   
269   7NH1GNJ4T2U4.BARQXH2A2PBW@mimefactory.state.tld   
342  G3SMNMV4T2U4.QBIQD41RBGS23@mimefactory.state.tld   
265  CLB3YPQ4T2U4.HONQ0DPSWVMN1@mimefactory.state.tld   
250  ZFT3B9E6T2U4.B4PYCHZIKGX23@mimefactory.state.tld   
259  VF9WHMQ4T2U4.HDJ31PQNP8R21@mimefactory.state.tld   
275   MSWE0J65T2U4.AVI51H7W6MLQ@mimefactory.state.tld   
62    X4JOMV65T2U4.KVM0VD0DCGN3@mimefactory.state.tld   
120   I8V50M76T2U4.QZF05G5VAPKE

In [73]:
#Create a training set and test set, 80% 20%
train_df, test_df = train_test_split(non_empty_df, train_size = 0.8, random_state=44)
class_labels_training = list(train_df['personal'])
class_labels_test = list(test_df['personal'])
value_counts = nltk.FreqDist(class_labels_training)


In [74]:
#Instantiate a stemmer and a tokenizer to preprocess the email text

snowballStemmer = SnowballStemmer("english", ignore_stopwords=True)
def preprocess(text):
    no_punctuation_text = ''
    if (type(text)== str):
        lower_text = text.lower()
        no_punctuation_text = lower_text.translate({ord(c):'' for c in string.punctuation})
    return no_punctuation_text

def myTokenize(text):
    global snowballStemmer
    tokens = []
    cleaned = preprocess(text)
    tokens = nltk.word_tokenize(cleaned)
    filtered = [w for w in tokens if not w in stopwords.words('english')]
    stemmed = [w for w in map(snowballStemmer.stem, filtered)]
    return stemmed
    
tokens = myTokenize(train_df['body'].iloc[0])
count = Counter(tokens)
print(count.most_common(10))

[('receptor', 35), ('color', 33), ('odor', 24), ('differ', 23), ('mixtur', 23), ('percept', 23), ('space', 20), ('similar', 19), ('bee', 18), ('light', 18)]


In [75]:
##Instantiate a TFidf vectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, encoding='utf-8', 
                             max_df=0.5, tokenizer=myTokenize)

In [76]:
#This step can take a while
train_X = vectorizer.fit_transform(train_df['body'])


In [77]:
##Save feature names in a separate list
feature_names = vectorizer.get_feature_names()

In [78]:
#From http://fastml.com/classifying-text-with-bag-of-words-a-tutorial/
#Create another matrix of tfidf scores for the documents in the test set
test_X = vectorizer.transform(test_df['body'])

In [79]:
#Create a logistic regression model
#From same tutorial https://github.com/zygmuntz/classifying-text/blob/master/bow_predict.py
model_lr = LR()
model_lr.fit(train_X, class_labels_training)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [80]:
#Try a naive bayes classifier instead
model_nb = MultinomialNB(alpha=0.05)
model_nb.fit(train_X, class_labels_training)

MultinomialNB(alpha=0.05, class_prior=None, fit_prior=True)

In [81]:
#Predict probability of class membership
p = model_lr.predict_proba( test_X )[:,1]
model_lr.score( test_X, class_labels_test)


0.93333333333333335

In [82]:
p2 = model_nb.predict( test_X )
model_nb.score( test_X, class_labels_test)

0.93333333333333335

In [83]:
results = [(class_labels_test[i], p2[i]) for i in range(0,len(p2))]
len(results)
results[0:20]

[(False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (True, False),
 (False, False),
 (False, False),
 (False, False)]

In [85]:
#Write out results
output_df = pd.DataFrame( data = {'messageId': test_df['messageId'], 
                                  'personal': test_df['personal'], 
                                  'logistic': p })


In [91]:
print("Output dataframe length: {}\n".format(len(output_df)))
THRESHOLD = .23
output_df['prediction'] = (output_df['logistic'] >= THRESHOLD)
print("DF columns:{}\n".format(", ".join(output_df.columns)))

##If 'prediction' matches 'about_fraud' then the classifier got it right
output_df.head(30)

Output dataframe length: 15

DF columns:logistic, messageId, personal, prediction



,logistic,messageId,personal,prediction
44,0.094744,YLEYPW75T2U4.D08XWS881YX41@mimefactory.state.tld,False,False
196,0.090744,TNVK9AI6T2U4.O75AXWYTCRWK3@mimefactory.state.tld,False,False
62,0.081948,X4JOMV65T2U4.KVM0VD0DCGN3@mimefactory.state.tld,False,False
330,0.096646,HP0CTGJ6T2U4.26RQ7U8N1SJN2@mimefactory.state.tld,False,False
228,0.077052,5C9ZDVG4T2U4.3KSXBPZG7GED2@mimefactory.state.tld,False,False
105,0.095078,ST3QAZ86T2U4.19EHJDZXNXAI3@mimefactory.state.tld,False,False
293,0.097110,7JBYG906T2U4.3PH5BEQFHY101@mimefactory.state.tld,False,False
341,0.078892,XDKEROS5T2U4.J7JEMQMLVMTK3@mimefactory.state.tld,False,False
0,0.094604,AI0QNEC5T2U4.LINXNFAF2B773@mimefactory.state.tld,False,False
177,0.083880,KGH19W25T2U4.LJI1H9C4UU873@mimefactory.state.tld,False,False


In [88]:
#Now let's see how well this model did, true positives, false positives, etc
def accuracy(tp, tn, fp, fn):
    return ((tp + tn)/(tp + tn + fp + fn))

def error_rate(tp, tn, fp, fn):
    return ((fp + fn)/ (tp + tn + fp + fn))

true_positives = len(output_df.loc[(output_df['personal'] == True) & (output_df['prediction'] == True)])
false_positives = len(output_df.loc[(output_df['personal'] == False) & (output_df['prediction'] == True)])
true_negatives = len(output_df.loc[(output_df['personal'] == False) & (output_df['prediction'] == False)])
false_negatives = len(output_df.loc[(output_df['personal'] == True) & (output_df['prediction'] == False)])
print("Results\nTrue Positives\tTrue_Negatives\tFalse_Positives\False_Negatives\n")
print("\t".join(map(str, [true_positives, true_negatives, false_positives, false_negatives])))

print("Classifier Accuracy: {}\n".format(accuracy(true_positives, true_negatives, 
                                                  false_positives, false_negatives)))
print("Classifier Error Rate: {}\n".format(error_rate(true_positives, true_negatives,
                                                      false_positives, false_negatives)))

Results
True Positives	True_Negatives	False_Positives\False_Negatives

0	14	0	1
Classifier Accuracy: 0.9333333333333333

Classifier Error Rate: 0.06666666666666667



In [89]:
from sklearn import metrics
print(metrics.classification_report(class_labels_test, p2))

             precision    recall  f1-score   support

      False       0.93      1.00      0.97        14
       True       0.00      0.00      0.00         1

avg / total       0.87      0.93      0.90        15



C:\Users\embicks\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [90]:
#What are the most informative features in this test?
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

show_most_informative_features(vectorizer, model_nb, 25)

	-10.0281	0              		-8.4351	lger           
	-10.0281	00             		-8.6678	citi           
	-10.0281	00001          		-8.6758	french         
	-10.0281	00002          		-8.6951	paint          
	-10.0281	00003          		-8.7458	berlin         
	-10.0281	00004          		-8.7494	war            
	-10.0281	00008          		-8.7586	moma           
	-10.0281	00016          		-8.8004	modern         
	-10.0281	0002           		-8.8221	machineri      
	-10.0281	000475         		-8.8226	worker         
	-10.0281	00058          		-8.8496	machin         
	-10.0281	0007           		-8.8999	capit          
	-10.0281	0008           		-8.9188	centuri        
	-10.0281	00082          		-8.9288	women          
	-10.0281	001            		-8.9377	soldier        
	-10.0281	001054         		-8.9405	communist      
	-10.0281	0013           		-8.9415	democrat       
	-10.0281	00131          		-8.9461	feel           
	-10.0281	00143          		-8.9521	factori        
	-10.0281	002            		-8.9